<a href="https://colab.research.google.com/github/SauloHenriqueAguiar/backtestBacktrading/blob/main/SharpeMAcrossover.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yahoo_fin

     |████████████████████████████████| 81 kB 8.7 MB/s 
     |████████████████████████████████| 83 kB 761 kB/s 
     |████████████████████████████████| 127 kB 58.2 MB/s 
     |████████████████████████████████| 103 kB 57.3 MB/s 
  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-py3-none-any.whl size=13502 sha256=0d3c5e171227e95e08d72e0c9be79396fe9b89b3c17df4569148e420394ac57b
  Stored in directory: /root/.cache/pip/wheels/ed/f7/62/50ab6c9a0b5567267ab76a9daa9d06315704209b2c5d032031
  Created wheel for parse: filename=parse-1.19.0-py3-none-any.whl size=24591 sha256=dd417b95cd1fe2101935a21e3737cd3c2628f3b1b51fd3892ee1d8efbe059238
  Stored in directory: /root/.cache/pip/wheels/9c/aa/cc/f2228050ccb40f22144b073f15a2c84f11204f29fc0dce028e
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6065 sha256=280c52af83aa827396a69cd5bd79a81afe5a0ca66d0fc814cc59d3b986fd1a69
  Stored in directory: /root/.cache/pip/wheels/73/ad/a4/0dff4a6ef231fc0dfa12ffbac2a36ce

In [2]:
#instala biblioteca yfinance
!pip install yfinance --upgrade --no-cache-dir

     |████████████████████████████████| 6.3 MB 37.7 MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.64-py2.py3-none-any.whl size=24109 sha256=4d795abd8c72ea4c3c0db3eb1f20ca056f3e79065cd8b3a1557f585923e9404b
  Stored in directory: /tmp/pip-ephem-wheel-cache-bpxl_zee/wheels/86/fe/9b/a4d3d78796b699e37065e5b6c27b75cff448ddb8b24943c288
Successfully built yfinance
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [3]:
#instala request
!pip install requests_html

In [30]:
# Instalar Backtrader
!pip install backtrader

In [42]:
!pip install backtrader[plotting]

In [43]:
# Obtenha os pacotes necessários

from datetime import datetime
import backtrader as bt
from IPython.display import display, Image
import yfinance as yf
import backtrader.analyzers as btanalyzers
import backtrader.feeds as btfeeds
import backtrader.strategies as btstrats

In [44]:
from yahoo_fin.stock_info import get_data
bitcoin_weekly = get_data("BTC-USD", start_date="12/04/2018", end_date="03/11/2021", index_as_date = True, interval="1wk")
bitcoin_weekly

,open,high,low,close,adjclose,volume,ticker
2018-12-03,3886.294922,4075.627686,3280.228760,3614.234375,3614.234375,33296896714,BTC-USD
2018-12-10,3612.046387,3647.332520,3191.303467,3252.839111,3252.839111,29869247431,BTC-USD
2018-12-17,3253.123047,4198.429688,3253.123047,3998.980225,3998.980225,46021506218,BTC-USD
2018-12-24,4000.331787,4271.792480,3642.632080,3865.952637,3865.952637,39249734918,BTC-USD
2018-12-31,3866.839111,4093.297363,3707.231201,4076.632568,4076.632568,34343716582,BTC-USD
...,...,...,...,...,...,...,...
2021-02-08,38886.828125,49487.640625,38076.324219,48717.289062,48717.289062,580021243858,BTC-USD
2021-02-15,48696.535156,58330.570312,46347.476562,57539.945312,57539.945312,470533297170,BTC-USD
2021-02-22,57532.738281,57533.390625,43241.617188,45137.769531,45137.769531,766679775805,BTC-USD
2021-03-01,45159.503906,52535.136719,45115.093750,51206.691406,51206.691406,333113779409,BTC-USD


In [45]:
bitcoin_weekly.to_csv('/content/bitcoin.csv')


In [54]:
# Instantiate Cerebro engine
cerebro = bt.Cerebro()

In [55]:
#Add data feed to Cerebro
data = bt.feeds.YahooFinanceCSVData(dataname='bitcoin.csv')
cerebro.adddata(data)

In [56]:
class MAcrossover(bt.Strategy): 
    # Moving average parameters
    params = (('pfast',5),('pslow',52),)

    def log(self, txt, dt=None):
        dt = dt or self.datas[0].datetime.date(0)
        print(f'{dt.isoformat()} {txt}') # Comment this line when running optimization

    def __init__(self):
        self.dataclose = self.datas[0].close        
		    # Order variable will contain ongoing order details/status
        self.order = None
        # Instantiate moving averages
        self.slow_sma = bt.indicators.MovingAverageSimple(self.datas[0], period=self.params.pslow)
        self.fast_sma = bt.indicators.MovingAverageSimple(self.datas[0], period=self.params.pfast)
    
    def notify_order(self, order):
      if order.status in [order.Submitted, order.Accepted]:
      # An active Buy/Sell order has been submitted/accepted - Nothing to do
        return
      # Check if an order has been completed
      # Attention: broker could reject order if not enough cash
      if order.status in [order.Completed]:
        if order.isbuy():
          self.log(f'BUY EXECUTED, {order.executed.price:.2f}')
        elif order.issell():
          self.log(f'SELL EXECUTED, {order.executed.price:.2f}')
        self.bar_executed = len(self)
      elif order.status in [order.Canceled, order.Margin, order.Rejected]:
        self.log('Order Canceled/Margin/Rejected')
      # Reset orders
      self.order = None
    

    def next(self):
      if self.order:
        return
      if not self.position:
        # We are not in the market, look for a signal to OPEN trades          
        #If the 20 SMA is above the 50 SMA
        if self.fast_sma[0] > self.slow_sma[0] and self.fast_sma[-1] < self.slow_sma[-1]:
          self.log(f'BUY CREATE {self.dataclose[0]:2f}')
          # Keep track of the created order to avoid a 2nd order
          self.order = self.buy()
        #Otherwise if the 20 SMA is below the 50 SMA   
        elif self.fast_sma[0] < self.slow_sma[0] and self.fast_sma[-1] > self.slow_sma[-1]:
          self.log(f'SELL CREATE {self.dataclose[0]:2f}')
          # Keep track of the created order to avoid a 2nd order
          self.order = self.sell()
      else:
        # We are already in the market, look for a signal to CLOSE trades
        if len(self) >= (self.bar_executed + 5):
          self.log(f'CLOSE CREATE {self.dataclose[0]:2f}')
          self.order = self.close()

In [57]:
# Add strategy to Cerebro
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe_ratio')
cerebro.optstrategy(MAcrossover, pfast=range(5, 20), pslow=range(50, 100)) 

In [58]:
# Default position size
cerebro.addsizer(bt.sizers.SizerFix, stake=3)

In [59]:


if __name__ == '__main__':
    optimized_runs = cerebro.run()

    final_results_list = []
    for run in optimized_runs:
        for strategy in run:
            PnL = round(strategy.broker.get_value() - 10000,2)
            sharpe = strategy.analyzers.sharpe_ratio.get_analysis()
            final_results_list.append([strategy.params.pfast, 
                strategy.params.pslow, PnL, sharpe['sharperatio']])

    sort_by_sharpe = sorted(final_results_list, key=lambda x: x[3], 
                             reverse=True)
    for line in sort_by_sharpe[:5]:
        print(line)

2019-12-16 SELL CREATE 7511.590000
2019-12-16 SELL CREATE 7511.590000
2019-12-23 SELL EXECUTED, 7508.90
2019-12-23 SELL EXECUTED, 7508.90
2020-01-27 CLOSE CREATE 9344.370000
2020-02-03 BUY EXECUTED, 9344.68
2020-01-27 CLOSE CREATE 9344.370000
2020-03-09 SELL CREATE 5392.310000
2020-03-16 SELL EXECUTED, 5385.23
2020-02-03 BUY EXECUTED, 9344.68
2020-04-19 CLOSE CREATE 7679.870000
2020-04-26 Order Canceled/Margin/Rejected
2020-03-09 SELL CREATE 5392.310000
2020-03-16 SELL EXECUTED, 5385.23
2020-04-26 CLOSE CREATE 8897.470000
2020-04-19 CLOSE CREATE 7679.870000
2020-05-03 Order Canceled/Margin/Rejected
2020-04-26 Order Canceled/Margin/Rejected
2020-05-03 CLOSE CREATE 8756.430000
2020-05-10 Order Canceled/Margin/Rejected
2020-05-10 CLOSE CREATE 9670.740000
2020-04-26 CLOSE CREATE 8897.470000
2020-05-17 Order Canceled/Margin/Rejected
2020-05-17 CLOSE CREATE 8790.370000
2020-05-24 Order Canceled/Margin/Rejected
2020-05-24 CLOSE CREATE 9461.060000
2020-05-03 Order Canceled/Margin/Rejected
2020

AttributeError: ignored